### Libraries

In [63]:
import pandas as pd
%matplotlib inline

### Load Data

In [64]:
df_train = pd.read_csv('./tanzania_water_training_set_values.csv')
df_train_labels = pd.read_csv('./tanzania_water_training_set_labels.csv')
df_test = pd.read_csv('./tanzania_water_test_set_values.csv')

### Initial Exploration

#### View Split of Labels

In [66]:
df_train_labels['status_group'].value_counts() / df_train_labels.shape[0]

functional                 0.543081
non functional             0.384242
functional needs repair    0.072677
Name: status_group, dtype: float64

#### View Null Labels

In [67]:
df_train.isnull().sum()

id                           0
amount_tsh                   0
date_recorded                0
funder                    3635
gps_height                   0
installer                 3655
longitude                    0
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                 371
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting            3334
recorded_by                  0
scheme_management         3877
scheme_name              28166
permit                    3056
construction_year            0
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_quality                0
quality_

#### View data types and select object columns

In [68]:
df_train.dtypes

id                         int64
amount_tsh               float64
date_recorded             object
funder                    object
gps_height                 int64
installer                 object
longitude                float64
latitude                 float64
wpt_name                  object
num_private                int64
basin                     object
subvillage                object
region                    object
region_code                int64
district_code              int64
lga                       object
ward                      object
population                 int64
public_meeting            object
recorded_by               object
scheme_management         object
scheme_name               object
permit                    object
construction_year          int64
extraction_type           object
extraction_type_group     object
extraction_type_class     object
management                object
management_group          object
payment                   object
payment_ty

In [69]:
object_columns = [column for column in df_train.columns if df_train[column].dtype == object]

In [70]:
for column in object_columns:
    print(column)
    print(df_train[column].value_counts())
    print('\n')

date_recorded
2011-03-15    572
2011-03-17    558
2013-02-03    546
2011-03-14    520
2011-03-16    513
2011-03-18    497
2011-03-19    466
2013-02-04    464
2013-01-29    459
2011-03-04    458
2013-02-14    444
2013-01-24    435
2011-03-05    434
2013-02-15    429
2013-03-15    428
2011-03-11    426
2013-01-30    421
2013-02-16    418
2011-03-23    417
2011-03-09    416
2013-01-18    409
2013-02-26    391
2011-03-30    391
2011-03-24    381
2013-03-19    381
2013-02-13    380
2013-01-23    379
2011-03-12    379
2011-03-03    378
2013-01-28    376
             ... 
2011-09-20      1
2004-03-06      1
2011-09-15      1
2013-12-01      1
2011-09-21      1
2004-04-05      1
2004-07-01      1
2011-09-08      1
2011-09-16      1
2004-04-01      1
2011-09-06      1
2011-09-14      1
2011-09-13      1
2013-01-06      1
2011-09-12      1
2011-09-11      1
2004-01-07      1
2011-09-28      1
2002-10-14      1
2011-09-26      1
2011-09-01      1
2011-09-05      1
2011-09-27      1
2011-09-17    

Name: management, dtype: int64


management_group
user-group    52490
commercial     3638
parastatal     1768
other           943
unknown         561
Name: management_group, dtype: int64


payment
never pay                25348
pay per bucket            8985
pay monthly               8300
unknown                   8157
pay when scheme fails     3914
pay annually              3642
other                     1054
Name: payment, dtype: int64


payment_type
never pay     25348
per bucket     8985
monthly        8300
unknown        8157
on failure     3914
annually       3642
other          1054
Name: payment_type, dtype: int64


water_quality
soft                  50818
salty                  4856
unknown                1876
milky                   804
coloured                490
salty abandoned         339
fluoride                200
fluoride abandoned       17
Name: water_quality, dtype: int64


quality_group
good        50818
salty        5195
unknown      1876
milky         804
colored 

#### Delete some object related columns and encode the rest

In [71]:
columns_to_exclude = ['funder', 'installer', 'wpt_name', 'subvillage', 'region', 'lga', 'ward', 'recorded_by', 'scheme_name', 'extraction_type_group', 'extraction_type', 'management', 'payment_type', 'water_quality', 'quantity_group', 'source', 'source_class', 'waterpoint_type']
columns_to_encode = ['basin', 'scheme_management', 'extraction_type_class', 'management_group', 'payment', 'quality_group', 'quantity', 'source_type', 'waterpoint_type_group']

In [72]:
df_train_clean = df_train.drop(columns_to_exclude, axis = 1)

In [73]:
print(df_train.shape)
print(df_train_clean.shape)

(59400, 40)
(59400, 22)


In [74]:
for column in columns_to_encode:
    dummy_columns = pd.get_dummies(df_train_clean[column], prefix = column + '_') # e
    df_train_clean = pd.concat([df_train_clean, dummy_columns], axis = 1)
    df_train_clean = df_train_clean.drop(column, axis = 1)

In [75]:
print(df_train_clean.shape)

(59400, 77)


#### Add some date features

In [77]:
df_train_clean['date_recorded'] = pd.to_datetime(df_train_clean['date_recorded'])
df_train_clean['year_recorded'] = df_train_clean['date_recorded'].dt.year
df_train_clean['month_recorded'] = df_train_clean['date_recorded'].dt.month

In [78]:
df_train_clean = df_train_clean.drop(['date_recorded'], axis = 1)

#### Save Cleaned Dataset

In [79]:
df_train_clean.to_csv('clean_training_set.csv', index = False)